In [ ]:
import cv2
import numpy as np
from skimage.feature import local_binary_pattern
from skimage.feature import greycomatrix, greycoprops
from skimage import io, color
import matplotlib.pyplot as plt
import pandas as pd


# Function to extract features from an image
def extract_image_features(image):
    features = {}
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # 1. Color Histogram (RGB)
    for i, channel in enumerate(["R", "G", "B"]):
        hist = cv2.calcHist([image], [i], None, [256], [0, 256])
        features[f"{channel}_Mean"] = np.mean(hist)
        features[f"{channel}_Std"] = np.std(hist)

    # 2. Gray-Level Co-Occurrence Matrix (GLCM)
    glcm = greycomatrix(gray, [1], [0], levels=256, symmetric=True, normed=True)
    features["GLCM_Contrast"] = greycoprops(glcm, "contrast")[0, 0]
    features["GLCM_Homogeneity"] = greycoprops(glcm, "homogeneity")[0, 0]

    # 3. Local Binary Pattern (LBP)
    lbp = local_binary_pattern(gray, P=8, R=1, method="uniform")
    features["LBP_Mean"] = np.mean(lbp)

    # 4. Hu Moments
    moments = cv2.HuMoments(cv2.moments(gray)).flatten()
    for i in range(7):
        features[f"Hu_Moment_{i+1}"] = moments[i]

    # 5. Histogram of Oriented Gradients (HOG)
    hog = cv2.HOGDescriptor()
    h = hog.compute(gray)
    features["HOG_Mean"] = np.mean(h)

    # 6. Edge Detection (Canny)
    edges = cv2.Canny(gray, 100, 200)
    features["Edge_Mean"] = np.mean(edges)

    # 7. Gabor Filter
    gabor_kernel = cv2.getGaborKernel(
        (21, 21), 8.0, np.pi / 4, 10.0, 0.5, 0, ktype=cv2.CV_32F
    )
    gabor_response = cv2.filter2D(gray, cv2.CV_8UC3, gabor_kernel)
    features["Gabor_Mean"] = np.mean(gabor_response)

    # 8. Fourier Transform
    f_transform = np.fft.fft2(gray)
    f_shift = np.fft.fftshift(f_transform)
    magnitude_spectrum = 20 * np.log(np.abs(f_shift))
    features["Fourier_Mean"] = np.mean(magnitude_spectrum)

    # 9. Discrete Cosine Transform (DCT)
    dct = cv2.dct(np.float32(gray))
    features["DCT_Mean"] = np.mean(dct)

    # 10. Zernike Moments (via OpenCV Hu Moments as approximation)
    features["Zernike_Mean"] = np.mean(moments)

    return features


# Function to generate random transformations (rotation, flip, etc.)
def random_transform(image):
    # Randomly rotate the image by 0-360 degrees
    angle = np.random.randint(0, 360)
    rows, cols = image.shape[:2]
    M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
    rotated_image = cv2.warpAffine(image, M, (cols, rows))

    # Randomly flip the image
    flip = np.random.choice([0, 1, -1])  # 0: vertical, 1: horizontal, -1: both
    flipped_image = cv2.flip(rotated_image, flip)

    return flipped_image


# Example usage
if __name__ == "__main__":
    image_path = "./DSC_1205-A_1.jpg"  # Replace with your image file path

    # Load the image
    image = cv2.imread(image_path)

    # List to store features
    features_list = []

    # Generate 100 rows of features by applying random transformations
    for _ in range(100):
        transformed_image = random_transform(image)
        features = extract_image_features(transformed_image)
        features_list.append(features)

    # Convert to DataFrame
    image_df = pd.DataFrame(features_list)

    # Display the DataFrame
    print(image_df)

    # Save features to CSV
    image_df.to_csv("image_features_dataset.csv", index=False)